# 01 Hent valgresultater

Denne notesbog tillader os at hente valgresultater til kommunal- og regionsrådsvalget 2025 fra en fjernserver hos [valg.dk](https://valg.dk/data-eksport).

Vi bruger `paramiko` til at oprette en SFTP-forbindelse og hente data.

Data bliver downloaded og struktureret i mapper baseret på valgtype og datakategori.

## Setup
Inden vi går i gang, skal vi sikre os, at vi har de nødvendige biblioteker installeret. Hvis du ikke allerede har `paramiko` og `os`, bliver det installeret her:

In [30]:
!pip install -q paramiko


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip


Derefter importerer vi de nødvendige biblioteker

In [31]:
import pandas as pd
import paramiko
import os
import json

## Forbindelse til SFTP-server
Først sætter parametrene for SFTP-forbindelsen op. De kommer [herfra](https://valg.dk/assets/guides/sftp-guide-public/VALG%20-%20Vejledning%20til%20hent%20af%20valgdata%20(Offentligheden).pdf).

In [32]:
host = "data.valg.dk"
port = 22
username = "Valg"
password = "Valg"

Så forbinder vi til SFTP-serveren

In [33]:
transport = paramiko.Transport((host, port))
transport.connect(username=username, password=password)

Og genererer en SFTP-klient

In [34]:
sftp = paramiko.SFTPClient.from_transport(transport)

In [35]:
# list all files in the directory
files = sftp.listdir('/data/kommunalvalg-134-18-11-2025/')
print(files)

['geografi', 'public-key.pem', 'verifikation', 'Snitfladebeskrivelser', 'kandidat-data']


## Download af data

Her definerer vi to centrale funktioner, som hjælper os med at downloade data. Funktionerne kan genbruges til både kommunal- og regionsrådsvalget.

`download_files` henter filer fra en given fjernmappe og gemmer dem lokalt i en tilsvarende lokal mappe.

`download_folders` henter mapper fra en given fjernmappe og gemmer dem lokalt i en tilsvarende lokal mappe.

In [36]:
def download_files(sftp, remote_dir, local_dir, folder_name):
    files = sftp.listdir(remote_dir+"/"+folder_name)

    for file in files:   # download each file
        remote_file_path = remote_dir + "/" + folder_name + "/" + file
        local_file_path = os.path.join(local_dir, folder_name, file)
        os.makedirs(os.path.dirname(local_file_path), exist_ok=True)
        sftp.get(remote_file_path, local_file_path)


def download_folders(folders):
    for folder in folders:
        download_files(sftp, remote_path, local_path, folder)

### Download KV25

In [37]:
remote_path = "/data/kommunalvalg-134-18-11-2025/verifikation"
local_path = "data/raw/kv"

In [38]:
folders = ["valgresultater", "kandidat-data", "mandatfordeling", "valgdeltagelse"]
download_folders(folders)

### Download RV25

In [39]:
remote_path = "/data/regionsrådsvalg-134-18-11-2025/verifikation"
local_path = "data/raw/rv"

In [40]:
folders = ["valgresultater", "mandatfordeling", "valgdeltagelse"]
download_folders(folders)

In [41]:
# sftp.close()
# transport.close()
# print("Download complete.")